# Evaluating tfcoder completions

In [3]:
import os
import sys
from pathlib import Path

CURRENT_DIRECTORY = Path(os.getcwd())
ROOT_DIRECTORY = (CURRENT_DIRECTORY / "..").absolute().resolve()

print(f"Current directory: {CURRENT_DIRECTORY}")
print(f"Root directory: {ROOT_DIRECTORY}")

sys.path.append(str(ROOT_DIRECTORY))

Current directory: /home/ubuntu/arga-arc/tf_coder
Root directory: /home/ubuntu/arga-arc


In [4]:
import typing as t
import json
from pprint import pprint
from dataclasses import dataclass
import tensorflow as tf
import numpy as np
import math

In [7]:
class OutputJSON(t.TypedDict):
    task_id: str
    completions: t.List[str]
    coverage_percentage: float
    description: str
    tf_operators: t.Dict[str, int]
    total_covered: int
    total_in_target: int

class ExamplesJSON(t.TypedDict):
    inputs: str
    outputs: str

class TaskJSON(t.TypedDict):
    constants: str
    description: str
    name: str
    source: str
    target_program: str
    examples: ExamplesJSON

class TaskWithOutputJSON(t.TypedDict):
    constants: str
    description: str
    name: str
    source: str
    target_program: str
    examples: ExamplesJSON
    response: OutputJSON



OUTPUT_FILE = CURRENT_DIRECTORY / "tfcoder_dataset_with_completions.in_context_ordered_by_weight.json"
DATASET_FILE = CURRENT_DIRECTORY / "tfcoder_dataset.json"

TASK_WITH_RESPONSE: t.List[TaskWithOutputJSON] = json.loads(OUTPUT_FILE.read_text())
OUTPUT = [task["response"] if 'response' in task else None for task in TASK_WITH_RESPONSE]
DATASET: t.List[TaskJSON] = json.loads(DATASET_FILE.read_text())

print(f"Loaded {len(OUTPUT)} outputs and {len(DATASET)} tasks")

TASK_JSONS = {task["name"]: task for task in DATASET}
OUTPUTS = {output["task_id"]: output for output in OUTPUT if output is not None}

keys = list(OUTPUTS.keys())[0:5]
for key in keys:
    print(f"Task: {key}")
    pprint(TASK_JSONS[key])
    pprint(OUTPUTS[key])
    print()

Loaded 72 outputs and 72 tasks
Task: google_01
{'constants': '[]',
 'description': 'Convert index tensor into pairs for SparseTensor indexing',
 'examples': {'inputs': '[[0, 0, 0, 1, 3, 3],]',
              'outputs': '[[0, 0], [0, 1], [0, 2], [1, 0], [3, 0], [3, 1]]'},
 'name': 'google_01',
 'source': 'From an internal Google chat room, 09/07/2018',
 'target_program': 'tf.cast(tf.where(tf.sequence_mask(tf.math.bincount(in1))), '
                   'tf.int32)'}
{'completions': ['return tf.stack([tf.math.bincount(in1), in1], axis=1)',
                 'return tf.stack([tf.divide(in1, tf.shape(in1)[0]), '
                 'tf.mod(in1, tf.shape(in1)[0])], axis=1)',
                 'return tf.stack([tf.math.floor_div(in1, tf.constant(2, '
                 'dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, '
                 'dtype=tf.int32))], axis=1)',
                 'return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], '
                 'axis=1)',
                 'return tf.

In [8]:
@dataclass
class Example:
    inputs: t.List[np.ndarray]
    output: t.Union[np.ndarray, tf.SparseTensor]

    @classmethod
    def from_json(cls, examples: ExamplesJSON):
        try:
            evaluated_inputs = eval(examples["inputs"])
            if isinstance(evaluated_inputs, list):
                inputs = [np.array(i) for i in evaluated_inputs]
            else:
                inputs = [evaluated_inputs]
        except Exception as e:
            print(f"Error evaluating inputs: {e}")
            print(f"Inputs: {examples['inputs']}")
            raise e

        try:
            evaluated_outputs = eval(examples["outputs"])
            if isinstance(evaluated_outputs, list):
                outputs = np.array(evaluated_outputs)
            elif isinstance(evaluated_outputs, tf.SparseTensor):
                outputs = evaluated_outputs
            elif isinstance(evaluated_outputs, tf.Tensor):
                outputs = evaluated_outputs.numpy()
            else:
                outputs = evaluated_outputs
        except Exception as e:
            print(f"Error evaluating outputs: {e}")
            print(f"Outputs: {examples['outputs']}")
            raise e

        return cls(inputs, outputs)

SKIP_TASKS = [] # ["google_13", "stackoverflow_05", "stackoverflow_10", "stackoverflow_13"]

# for each task in dataset, show inputs and outputs
for task in DATASET:
    if task["name"] in SKIP_TASKS:
        continue
    print(f"Task name: {task['name']}")
    pprint(Example.from_json(task["examples"]))
    print()

Task name: google_01
Example(inputs=[array([0, 0, 0, 1, 3, 3])],
        output=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [3, 0],
       [3, 1]]))

Task name: google_02
Example(inputs=[array([[0., 1., 0., 0.],
       [0., 1., 1., 0.],
       [1., 1., 1., 1.]])],
        output=array([[0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.5 , 0.5 , 0.  ],
       [0.25, 0.25, 0.25, 0.25]]))

Task name: google_03
Example(inputs=[SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 1 1]
 [1 1 1]
 [1 1 2]], shape=(4, 3), dtype=int64), values=tf.Tensor([1. 1. 1. 1.], shape=(4,), dtype=float32), dense_shape=tf.Tensor([  2   2 800], shape=(3,), dtype=int64))],
        output=SparseTensor(indices=tf.Tensor(
[[0 0 0]
 [0 1 1]], shape=(2, 3), dtype=int64), values=tf.Tensor([1. 1.], shape=(2,), dtype=float32), dense_shape=tf.Tensor([  1   2 800], shape=(3,), dtype=int64)))

Task name: google_04
Example(inputs=[array([111, 112, 121, 122, 131, 132, 211, 212, 221, 222, 231, 232, 311,
       312, 

2024-03-27 21:49:58.656353: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [15]:
TEST_COMPLETIONS = [
"""
input = tf.constant([[1, 2, 3, 4], [5, 6, 7, 8]])

output = tf.repeat(input, repeats=6, axis=0)
"""
]

def evaluate_completion(completion: str) -> t.List[t.Union[np.ndarray, tf.SparseTensor]]:
    """
    evaluates the completion, returning any local variables with an 
    array-like value
    """
    
    locals_dict = {}
    try:
        exec(completion, None, locals_dict)
    except Exception as e:
        print(f"Error evaluating completion: {e}")
        print(f"Completion: {completion}")
        return []
    
    ans = []

    for key, value in locals_dict.items():
        try:
            if isinstance(value, np.ndarray) or isinstance(value, tf.SparseTensor):
                ans.append(value)
            elif isinstance(value, tf.Tensor):
                ans.append(value.numpy())
            elif isinstance(value, list):
                ans.append(np.array(value))
        except Exception as e:
            print(f"Error adding value: {e}")
            print(f"Key: {key}")
            print("value:")
            pprint(value)
            print(f"Type: {type(value)}")
    return ans

def matches_expected_value(actual: t.Union[np.ndarray, tf.SparseTensor], expected: t.Union[np.ndarray, tf.SparseTensor]) -> bool:
    if isinstance(actual, np.ndarray) and isinstance(expected, np.ndarray):
        return np.array_equal(actual, expected)
    elif isinstance(actual, tf.SparseTensor) and isinstance(expected, tf.SparseTensor):
        return tf.sparse.equal(actual, expected)
    else:
        return False

pprint([evaluate_completion(completion) for completion in TEST_COMPLETIONS])

SyntaxError: invalid syntax (<string>, line 2)

In [53]:
FIXUP_EXAMPLE_COMPLETIONS = [
    """weights = tf.range(0, tf.shape(in1)[1], 1)
in1_weighted = tf.multiply(in1, tf.cast(weights, tf.float32))
return tf.reduce_sum(in1_weighted, axis=1)""",
    """dense_in1 = tf.sparse.to_dense(in1[0], validate_indices=False)
     return tf.reduce_sum(tf.math.squared_difference(in1[1], dense_in1))""",
    """result = tf.zeros_like(in2, dtype=tf.int32)
counter = tf.constant([0])
for i in range(in2.shape[0]):
    for j in range(in2.shape[1]):
        if in2[i][j]:
            result[i,j].assign(in1[counter])
            counter += 1
return result""",
    """segments = tf.math.bincount(in1)
order = tf.argsort(in1, stable=True)
sorted_values = tf.gather(in2, order)
segment_marks = tf.cumsum(segments)

starts = tf.pad(segment_marks[:-1], [[1,0]])
ends = tf.pad(segment_marks[1:], [[0,1]])

segments = []
for i in range(len(starts)):
    segment = sorted_values[starts[i]:ends[i]]
    segments.append(segment)

return tf.concat(segments, 0)"""
]

def fixup_completion(prefix: str, completion: str) -> str:
    lines = completion.split("\n")

    def combine_lines_and_prefix(ls):
        return prefix + "\n" + "\n".join([f"   {line}" for line in ls])

    # if the completion contains a loop or conditional, keep indentation as is
    if "for " in completion or "while " in completion or "if " in completion:
        return combine_lines_and_prefix(lines)
    
    stripped_lines = [line.lstrip() for line in lines]
    return combine_lines_and_prefix(stripped_lines)

for completion in FIXUP_EXAMPLE_COMPLETIONS:
    print(fixup_completion("def transform():", completion))
    print()


def transform():
   weights = tf.range(0, tf.shape(in1)[1], 1)
   in1_weighted = tf.multiply(in1, tf.cast(weights, tf.float32))
   return tf.reduce_sum(in1_weighted, axis=1)

def transform():
   dense_in1 = tf.sparse.to_dense(in1[0], validate_indices=False)
   return tf.reduce_sum(tf.math.squared_difference(in1[1], dense_in1))

def transform():
   result = tf.zeros_like(in2, dtype=tf.int32)
   counter = tf.constant([0])
   for i in range(in2.shape[0]):
       for j in range(in2.shape[1]):
           if in2[i][j]:
               result[i,j].assign(in1[counter])
               counter += 1
   return result

def transform():
   segments = tf.math.bincount(in1)
   order = tf.argsort(in1, stable=True)
   sorted_values = tf.gather(in2, order)
   segment_marks = tf.cumsum(segments)
   
   starts = tf.pad(segment_marks[:-1], [[1,0]])
   ends = tf.pad(segment_marks[1:], [[0,1]])
   
   segments = []
   for i in range(len(starts)):
       segment = sorted_values[starts[i]:ends[i]]
       segment

In [54]:
@dataclass
class Task:
    name: str
    description: str
    target_program: str
    examples: Example
    completions: t.List[str]

    @property
    def completion_prefix(self) -> str:
        formals_str = [f"in{i+1}" for i in range(len(self.examples.inputs))]
        return f"def transform({', '.join(formals_str)}):"

    @classmethod
    def from_json(cls, task: TaskJSON, output: OutputJSON):
        assert task["name"] == output["task_id"]
        return cls(
            task["name"],
            task["description"],
            task["target_program"],
            Example.from_json(task["examples"]),
            output["completions"]
        )

    def num_correct_completions(self) -> int:
        ans = 0
        for completion in self.completions:
            # evaluated = evaluate_completion(self.completion_prefix + completion)
            evaluable_completion = fixup_completion(self.completion_prefix, completion)
            try:
                exec(evaluable_completion)
                result = locals()["transform"](*self.examples.inputs)
                if matches_expected_value(result, self.examples.output):
                    ans += 1
            except Exception as e:
                print(completion)
                print(str(e))
                print()
                continue
        return ans

TASKS = {
    task["name"]: Task.from_json(task, OUTPUTS[task["name"]])
    for task in DATASET
    if task["name"] in OUTPUTS
}

# pprint 5 tasks
keys = list(TASKS.keys())
for key in keys:
    print(f"Task: {key}")
    # pprint(TASKS[key])
    pprint(TASKS[key].num_correct_completions())
    print()

Task: google_01
return tf.stack([tf.math.bincount(in1), in1], axis=1)
{{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [4] != values[1].shape = [6] [Op:Pack] name: stack

return tf.stack([tf.divide(in1, tf.shape(in1)[0]), tf.mod(in1, tf.shape(in1)[0])], axis=1)
module 'tensorflow' has no attribute 'mod'

return tf.stack([tf.math.floor_div(in1, tf.constant(2, dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, dtype=tf.int32))], axis=1)
module 'tensorflow._api.v2.math' has no attribute 'floor_div'

return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], axis=1)
module 'tensorflow' has no attribute 'floor_div'

return tf.stack([tf.math.floor_divide(in1, tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], axis=1)
module 'tensorflow._api.v2.math' has no attribute 'floor_divide'

return tf.stack([tf.math.floor_divide(in1, tf.reduce_max(in1) + 1), tf.math.floormod(in1, tf.reduce_max(in1) + 1)],

2024-03-27 23:58:53.498181: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:58:53.498852: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:58:53.499561: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:58:53.500530: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:58:53.501146: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:58:53.501738: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 :

0

Task: stackoverflow_08
0

Task: stackoverflow_09
_, idx, count = tf.unique_with_counts(in1)
return tf.gather(idx, in1)
{{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0] = 37 is not in [0, 8) [Op:GatherV2] name: 

_, idx = tf.unique_with_counts(in1)
return idx
too many values to unpack (expected 2)

_, idx = tf.unique_with_counts(in1)
 return tf.gather(idx, in1)
too many values to unpack (expected 2)

1

Task: stackoverflow_10
return tf.matmul(tf.expand_dims(in1, 0), tf.expand_dims(in2, -1))
{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,2,2], In[1]: [2,3,1] [Op:BatchMatMulV2] name: 

return tf.matmul(tf.expand_dims(in1,axis =2), tf.reshape(in2,[in1.shape[1],-1,1]))
{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [2,2,1], In[1]: [2,3,1] [Op:BatchMatMulV2] name: 

return tf

2024-03-27 23:58:53.940496: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:58:53.941473: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:58:53.942684: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:58:53.943494: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:58:53.945010: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indi

In [55]:
for task in TASKS.values():
    print(f"Task: {task.name}")
    print(f"GPT-4 Successfully solved: {task.num_correct_completions()}")
    print()

Task: google_01
return tf.stack([tf.math.bincount(in1), in1], axis=1)
{{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [4] != values[1].shape = [6] [Op:Pack] name: stack

return tf.stack([tf.divide(in1, tf.shape(in1)[0]), tf.mod(in1, tf.shape(in1)[0])], axis=1)
module 'tensorflow' has no attribute 'mod'

return tf.stack([tf.math.floor_div(in1, tf.constant(2, dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, dtype=tf.int32))], axis=1)
module 'tensorflow._api.v2.math' has no attribute 'floor_div'

return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], axis=1)
module 'tensorflow' has no attribute 'floor_div'

return tf.stack([tf.math.floor_divide(in1, tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], axis=1)
module 'tensorflow._api.v2.math' has no attribute 'floor_divide'

return tf.stack([tf.math.floor_divide(in1, tf.reduce_max(in1) + 1), tf.math.floormod(in1, tf.reduce_max(in1) + 1)],

2024-03-27 23:59:33.207396: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:33.207904: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:33.208442: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:33.209242: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:33.209744: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:33.210217: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 :

GPT-4 Successfully solved: 0

Task: stackoverflow_12
GPT-4 Successfully solved: 0

Task: stackoverflow_13
return tf.matmul(in1, tf.cast(in2, tf.int32))
cannot compute BatchMatMulV2 as input #1(zero-based) was expected to be a int64 tensor but is a int32 tensor [Op:BatchMatMulV2] name: 

return tf.matmul(in2, tf.cast(tf.transpose(in1), tf.int32))
cannot compute BatchMatMulV2 as input #1(zero-based) was expected to be a int64 tensor but is a int32 tensor [Op:BatchMatMulV2] name: 

GPT-4 Successfully solved: 0

Task: stackoverflow_14
GPT-4 Successfully solved: 0

Task: stackoverflow_15
GPT-4 Successfully solved: 0

Task: stackoverflow_16
GPT-4 Successfully solved: 0

Task: stackoverflow_17
return tf.expand_dims(in1, 1) * tf.ones((1, 2), dtype=tf.int32)
cannot compute Mul as input #1(zero-based) was expected to be a int64 tensor but is a int32 tensor [Op:Mul] name: 

return tf.expand_dims(in1, 1) * tf.ones([1,2], dtype=tf.int32)
cannot compute Mul as input #1(zero-based) was expected to be

2024-03-27 23:59:33.546820: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:33.547772: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:33.548952: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:33.549695: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:33.551119: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indi

In [56]:
EVALUATED_TASKS_FILE = CURRENT_DIRECTORY / "evaluated_tfcoder.json"

EVALUATED_TASKS_FILE.write_text(json.dumps(
    {
        task.name: task.num_correct_completions()
        for task in TASKS.values()
    }
))

return tf.stack([tf.math.bincount(in1), in1], axis=1)
{{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [4] != values[1].shape = [6] [Op:Pack] name: stack

return tf.stack([tf.divide(in1, tf.shape(in1)[0]), tf.mod(in1, tf.shape(in1)[0])], axis=1)
module 'tensorflow' has no attribute 'mod'

return tf.stack([tf.math.floor_div(in1, tf.constant(2, dtype=tf.int32)), tf.math.floormod(in1, tf.constant(2, dtype=tf.int32))], axis=1)
module 'tensorflow._api.v2.math' has no attribute 'floor_div'

return tf.stack([tf.floor_div(in1, 2), tf.mod(in1, 2)], axis=1)
module 'tensorflow' has no attribute 'floor_div'

return tf.stack([tf.math.floor_divide(in1, tf.shape(in1)[0]), tf.math.mod(in1, tf.shape(in1)[0])], axis=1)
module 'tensorflow._api.v2.math' has no attribute 'floor_divide'

return tf.stack([tf.math.floor_divide(in1, tf.reduce_max(in1) + 1), tf.math.floormod(in1, tf.reduce_max(in1) + 1)], axis=1)
module 

2024-03-27 23:59:36.106679: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:36.107236: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:36.107700: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:36.108502: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:36.108958: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 : INVALID_ARGUMENT: Expected a non-negative size, got -1
2024-03-27 23:59:36.109409: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at sparse_slice_op.cc:60 :

_, idx, count = tf.unique_with_counts(in1)
return tf.gather(idx, in1)
{{function_node __wrapped__GatherV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} indices[0] = 37 is not in [0, 8) [Op:GatherV2] name: 

_, idx = tf.unique_with_counts(in1)
return idx
too many values to unpack (expected 2)

_, idx = tf.unique_with_counts(in1)
 return tf.gather(idx, in1)
too many values to unpack (expected 2)

return tf.matmul(tf.expand_dims(in1, 0), tf.expand_dims(in2, -1))
{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [1,2,2], In[1]: [2,3,1] [Op:BatchMatMulV2] name: 

return tf.matmul(tf.expand_dims(in1,axis =2), tf.reshape(in2,[in1.shape[1],-1,1]))
{{function_node __wrapped__BatchMatMulV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Matrix size-incompatible: In[0]: [2,2,1], In[1]: [2,3,1] [Op:BatchMatMulV2] name: 

return tf.matmul(tf.expand_dims(in1, 0), tf.expand_dims(in2, -1))
{{function_node __wra

2024-03-27 23:59:36.544717: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:36.546524: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:36.547367: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:36.548175: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indices[5] = [5, 0] does not index into param shape [3,6], node name: GatherNd
2024-03-27 23:59:36.549818: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at gather_nd_op.cc:48 : INVALID_ARGUMENT: indi

1447